In [1]:
#| hide
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

In [2]:
import sys
import logging
import ipytest
import json
from pathlib import Path
import os

import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession, LocalPipelineSession
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import FrameworkProcessor
from sagemaker.image_uris import retrieve
from sagemaker.workflow.parameters import ParameterString
from sagemaker.workflow.steps import CacheConfig
from sagemaker.workflow.steps import TrainingStep
from sagemaker.inputs import TrainingInput
from sagemaker.tensorflow import TensorFlow
from sagemaker.workflow.parameters import ParameterInteger, ParameterFloat


CODE_FOLDER = Path("code")
CODE_FOLDER.mkdir(parents=True, exist_ok=True)
INFERENCE_CODE_FOLDER = CODE_FOLDER / "inference"
INFERENCE_CODE_FOLDER.mkdir(parents=True, exist_ok=True)

sys.path.extend([f"./{CODE_FOLDER}", f"./{INFERENCE_CODE_FOLDER}"])

DATA_FILEPATH = "penguins.csv"

ipytest.autoconfig(raise_on_error=True)

# By default, The SageMaker SDK logs events related to the default
# configuration using the INFO level. To prevent these from spoiling
# the output of this notebook cells, we can change the logging
# level to ERROR instead.
logging.getLogger("sagemaker.config").setLevel(logging.ERROR)

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/jerryobex/Library/Application Support/sagemaker/config.yaml


In [3]:

LOCAL_MODE = False

bucket = os.environ["BUCKET"]
role = os.environ["ROLE"]

S3_LOCATION = f"s3://{bucket}/penguins"

architecture = !(uname -m)
IS_APPLE_M_CHIP = architecture[0] == "arm64"


pipeline_session = PipelineSession(default_bucket=bucket) if not LOCAL_MODE else None

if LOCAL_MODE:
    config = {
        "session": LocalPipelineSession(default_bucket=bucket),
        "instance_type": "local",
        # We need to use a custom Docker image when we run the pipeline
        # in Local Model on an ARM64 machine.
        "image": "sagemaker-tensorflow-toolkit-local" if IS_APPLE_M_CHIP else None,
    }
else:
    config = {
        "session": pipeline_session,
        "instance_type": "ml.m5.xlarge",
        "image": None,
    }

config["framework_version"] = "2.11"
config["py_version"] = "py39"

In [4]:
import boto3

sagemaker_session = sagemaker.session.Session()
sagemaker_client = boto3.client("sagemaker")
iam_client = boto3.client("iam")
region = boto3.Session().region_name

USE_TUNING_STEP = not LOCAL_MODE
# USE_TUNING_STEP = False and not LOCAL_MODE

In [5]:
# | code: true
# | output: false

cache_config = CacheConfig(enable_caching=True, expire_after="15d")

# dataset_location = ParameterString(
#     name="dataset_location",
#     default_value=f"{S3_LOCATION}/data",
# )

dataset_location = ParameterString(
    name="dataset_location",
    default_value=f"{S3_LOCATION}/data",
)

new_prep_processor = False

processor = None

if new_prep_processor:
    # Specify the directory containing your `requirements.txt`
    processor = FrameworkProcessor(
        framework_version="1.2-1",
        estimator_cls=SKLearnProcessor,
        base_job_name="preprocess-data",
        image_uri=retrieve(framework='sklearn', version='1.2-1', region=config["session"].boto_region_name),
        command=["python3", "-m", "preprocessor"],
        instance_type=config["instance_type"],
        instance_count=2,
        role=role,
        sagemaker_session=config["session"],
    )

else:
    processor = SKLearnProcessor(
        base_job_name="preprocess-data",
        framework_version="1.2-1",
        # By default, a new account doesn't have access to `ml.m5.xlarge` instances.
        # If you haven't requested a quota increase yet, you can use an
        # `ml.t3.medium` instance type instead. This will work out of the box, but
        # the Processing Job will take significantly longer than it should have.
        # To get access to `ml.m5.xlarge` instances, you can request a quota
        # increase under the Service Quotas section in your AWS account.
        instance_type=config["instance_type"],
        instance_count=2,
        role=role,
        sagemaker_session=config["session"],
    )


preprocessing_step = ProcessingStep(
    name="preprocess-data",
    step_args=processor.run(
        code=f"{CODE_FOLDER}/preprocessor.py",
        inputs=[
            ProcessingInput(
                source=dataset_location, destination="/opt/ml/processing/input", s3_data_distribution_type="ShardedByS3Key"
            )
        ],
        outputs=[
            ProcessingOutput(
                output_name="train",
                source="/opt/ml/processing/train",
                destination=f"{S3_LOCATION}/preprocessing/train",
            ),
            ProcessingOutput(
                output_name="validation",
                source="/opt/ml/processing/validation",
                destination=f"{S3_LOCATION}/preprocessing/validation",
            ),
            ProcessingOutput(
                output_name="test",
                source="/opt/ml/processing/test",
                destination=f"{S3_LOCATION}/preprocessing/test",
            ),
            ProcessingOutput(
                output_name="model",
                source="/opt/ml/processing/model",
                destination=f"{S3_LOCATION}/preprocessing/model",
            ),
            ProcessingOutput(
                output_name="train-baseline",
                source="/opt/ml/processing/train-baseline",
                destination=f"{S3_LOCATION}/preprocessing/train-baseline",
            ),
            ProcessingOutput(
                output_name="test-baseline",
                source="/opt/ml/processing/test-baseline",
                destination=f"{S3_LOCATION}/preprocessing/test-baseline",
            ),
        ],
    ),
    cache_config=cache_config,
)


epochs = ParameterInteger(name="epochs", default_value=50)

estimator = TensorFlow(
    base_job_name="training",
    entry_point=f"{CODE_FOLDER}/train.py",
    # SageMaker will pass these hyperparameters as arguments
    # to the entry point of the training script.
    hyperparameters={
        "epochs": epochs,  # Referencing the pipeline parameter
        "batch_size": 32,
        # "learning_rate": 0.01,
        'learning_rate': ParameterFloat(0.01),
    },
    # SageMaker will track these metrics as part of the experiment
    # associated to this pipeline. The metric definitions tells
    # SageMaker how to parse the values from the Training Job logs.
    metric_definitions=[
        {"Name": "loss", "Regex": "loss: ([0-9\\.]+)"},
        {"Name": "accuracy", "Regex": "accuracy: ([0-9\\.]+)"},
        {"Name": "val_loss", "Regex": "val_loss: ([0-9\\.]+)"},
        {"Name": "val_accuracy", "Regex": "val_accuracy: ([0-9\\.]+)"},
    ],
    image_uri=config["image"],
    framework_version=config["framework_version"],
    py_version=config["py_version"],
    instance_type=config["instance_type"],
    instance_count=1,
    disable_profiler=True,
    sagemaker_session=config["session"],
    role=role,
)

train_model_step = TrainingStep(
    name="train-model",
    estimator=estimator,
    inputs={
        "train": TrainingInput(
            s3_data=preprocessing_step.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=preprocessing_step.properties.ProcessingOutputConfig.Outputs["validation"].S3Output.S3Uri,
            content_type="text/csv"
        )
    },
    # cache_config is optional, shown here as part of a complete example
    cache_config=cache_config  # Assuming cache_config is defined elsewhere
)

/Users/jerryobex/proj/mlschool/.venv/lib/python3.9/site-packages/sagemaker/workflow/pipeline_context.py:297: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [6]:
%%writefile {CODE_FOLDER}/evaluation_old.py
#| label: evaluation-script
#| echo: true
#| output: false
#| filename: evaluation.py
#| code-line-numbers: true

import json
import tarfile
import numpy as np
import pandas as pd

from pathlib import Path
from tensorflow import keras
from sklearn.metrics import accuracy_score, precision_score, recall_score

def evaluate(model_path, test_path, output_path):
    X_test = pd.read_csv(Path(test_path) / "test.csv")
    y_test = X_test.iloc[:, -1]  # Assuming the last column is the target
    island_columns = X_test.columns[-4:-1]  # Last 3 columns are one-hot encoded islands, excluding the target
    X_test.drop(X_test.columns[-1], axis=1, inplace=True)  # Drop target column only

    with tarfile.open(Path(model_path) / "model.tar.gz") as tar:
        tar.extractall(path=Path(model_path))
        
    model = keras.models.load_model(Path(model_path) / "001")
    
    predictions = np.argmax(model.predict(X_test), axis=-1)
    overall_accuracy = accuracy_score(y_test, predictions)
    print(f"Overall Test Accuracy: {overall_accuracy}")

    precision = precision_score(y_test, predictions, average=None, zero_division=0)
    recall = recall_score(y_test, predictions, average=None, zero_division=0)
    
    # Print overall precision and recall
    print("Overall Precision by Class:", precision)
    print("Overall Recall by Class:", recall)

    island_accuracies = {}
    for col in island_columns:
        island_mask = X_test[col] == 1
        y_island = y_test[island_mask]
        if len(y_island) > 0:
            island_predictions = np.argmax(model.predict(X_test[island_mask]), axis=-1)
            island_accuracy = accuracy_score(y_island, island_predictions)
            island_accuracies[col] = island_accuracy
            print(f"Accuracy for island {col}: {island_accuracy}")
        else:
            island_accuracies[col] = None
            print(f"No samples for {col}.")

    evaluation_report = {
        "metrics": {
            "accuracy": {"value": overall_accuracy},
            "island_accuracies": island_accuracies,
            "precision": {f"class_{i}": val for i, val in enumerate(precision)},
            "recall": {f"class_{i}": val for i, val in enumerate(recall)},
        },
    }
    
    Path(output_path).mkdir(parents=True, exist_ok=True)
    with open(Path(output_path) / "evaluation.json", "w") as f:
        f.write(json.dumps(evaluation_report))
        
if __name__ == "__main__":
    evaluate(
        model_path="/opt/ml/processing/model/", 
        test_path="/opt/ml/processing/test/",
        output_path="/opt/ml/processing/evaluation/"
    )


Overwriting code/evaluation_old.py


In [7]:
%%ipytest -s
#| code-fold: true
#| output: false

import os
import shutil
import tarfile
import pytest
import tempfile
import joblib

from preprocessor import preprocess
from train import train
from evaluation_old import evaluate


@pytest.fixture(scope="function", autouse=False)
def directory():
    directory = tempfile.mkdtemp()
    input_directory = Path(directory) / "input"
    input_directory.mkdir(parents=True, exist_ok=True)
    shutil.copy2(DATA_FILEPATH, input_directory / "data.csv")
    
    directory = Path(directory)
    
    preprocess(base_directory=directory)
    
    train(
        model_directory=directory / "model",
        train_path=directory / "train", 
        validation_path=directory / "validation",
        epochs=1
    )
    
    # After training a model, we need to prepare a package just like
    # SageMaker would. This package is what the evaluation script is
    # expecting as an input.
    with tarfile.open(directory / "model.tar.gz", "w:gz") as tar:
        tar.add(directory / "model" / "001", arcname="001")
        
    evaluate(
        model_path=directory, 
        test_path=directory / "test",
        output_path=directory / "evaluation",
    )

    yield directory / "evaluation"
    
    shutil.rmtree(directory)


def test_evaluate_generates_evaluation_report(directory):
    output = os.listdir(directory)
    assert "evaluation.json" in output


def test_evaluation_report_contains_accuracy(directory):
    with open(directory / "evaluation.json", 'r') as file:
        report = json.load(file)
        
    assert "metrics" in report
    assert "accuracy" in report["metrics"]
    assert "island_accuracies" in report["metrics"]
    assert "precision" in report["metrics"]
    assert "recall" in report["metrics"]
    

8/8 - 0s - loss: 1.2286 - accuracy: 0.2050 - val_loss: 1.2092 - val_accuracy: 0.1961 - 166ms/epoch - 21ms/step
2/2 [==============================] - 0s 995us/step
Validation accuracy: 0.19607843137254902
INFO:tensorflow:Assets written to: /var/folders/5k/bjy1b7pd2wxctw1dtvx0l2fc0000gn/T/tmpcf245y8e/model/001/assets
2/2 [==============================] - 0s 1ms/step
Overall Test Accuracy: 0.19607843137254902
Overall Precision by Class: [1.         0.20454545 0.        ]
Overall Recall by Class: [0.04347826 0.9        0.        ]
1/1 [==============================] - 0s 8ms/step
Accuracy for island 0.0: 0.0
1/1 [==============================] - 0s 7ms/step
Accuracy for island 1.0: 0.5555555555555556
1/1 [==============================] - 0s 7ms/step
Accuracy for island 0.0.1: 0.0
.8/8 - 0s - loss: 1.0576 - accuracy: 0.3473 - val_loss: 0.9865 - val_accuracy: 0.5490 - 149ms/epoch - 19ms/step
2/2 [==============================] - 0s 979us/step
Validation accuracy: 0.5490196078431373
INF

In [8]:
%%writefile {CODE_FOLDER}/evaluation.py
#| label: evaluation-script
#| echo: true
#| output: false
#| filename: evaluation.py
#| code-line-numbers: true

import json
import tarfile
import numpy as np
import pandas as pd

from pathlib import Path
from tensorflow import keras
from sklearn.metrics import accuracy_score, precision_score, recall_score

def load_and_evaluate_model(model_path, X_test, y_test, island_columns):
    with tarfile.open(Path(model_path) / "model.tar.gz") as tar:
        tar.extractall(path=Path(model_path))
        
    model = keras.models.load_model(Path(model_path) / "001")
    
    predictions = np.argmax(model.predict(X_test), axis=-1)
    overall_accuracy = accuracy_score(y_test, predictions)
    
    precision = precision_score(y_test, predictions, average=None, zero_division=0)
    recall = recall_score(y_test, predictions, average=None, zero_division=0)
    
    island_accuracies = {}
    for col in island_columns:
        island_mask = X_test[col] == 1
        y_island = y_test[island_mask]
        if len(y_island) > 0:
            island_predictions = np.argmax(model.predict(X_test[island_mask]), axis=-1)
            island_accuracy = accuracy_score(y_island, island_predictions)
            island_accuracies[col] = island_accuracy
        else:
            island_accuracies[col] = None
    
    return overall_accuracy, island_accuracies, precision, recall

def evaluate_models(trained_model_path, tuned_model_path, test_path, output_path):
    X_test = pd.read_csv(Path(test_path) / "test.csv")
    y_test = X_test.iloc[:, -1]  # Assuming the last column is the target
    island_columns = X_test.columns[-4:-1]  # Last 3 columns are one-hot encoded islands, excluding the target
    X_test.drop(X_test.columns[-1], axis=1, inplace=True)  # Drop target column only
    
    trained_model_accuracy, trained_island_accuracies, trained_precision, trained_recall = load_and_evaluate_model(trained_model_path, X_test, y_test, island_columns)
    tuned_model_accuracy, tuned_island_accuracies, tuned_precision, tuned_recall = load_and_evaluate_model(tuned_model_path, X_test, y_test, island_columns)
    
    best_model, best_accuracy = ("tuned", tuned_model_accuracy) if tuned_model_accuracy > trained_model_accuracy else ("trained", trained_model_accuracy)
    best_model_path = tuned_model_path if best_model == "tuned" else trained_model_path
    
    evaluation_report = {
        "best_model": best_model,
        "best_model_accuracy": best_accuracy,
        "best_model_path": str(best_model_path),
        "comparison": {
            "trained_model_accuracy": trained_model_accuracy,
            "tuned_model_accuracy": tuned_model_accuracy,
        }
    }
    
    Path(output_path).mkdir(parents=True, exist_ok=True)
    with open(Path(output_path) / "evaluation.json", "w") as f:
        f.write(json.dumps(evaluation_report))

if __name__ == "__main__":
    evaluate_models(
        trained_model_path="/opt/ml/processing/model/",
        tuned_model_path="/opt/ml/processing/model/",
        test_path="/opt/ml/processing/test/",
        output_path="/opt/ml/processing/evaluation/"
    )


Overwriting code/evaluation.py


In [9]:
%%ipytest -s
#| code-fold: true
#| output: false

import os
import shutil
import tarfile
import pytest
import tempfile
import joblib

from preprocessor import preprocess
from train import train
from evaluation import evaluate_models


@pytest.fixture(scope="function", autouse=False)
def directory():
    directory = tempfile.mkdtemp()
    input_directory = Path(directory) / "input"
    input_directory.mkdir(parents=True, exist_ok=True)
    shutil.copy2(DATA_FILEPATH, input_directory / "data.csv")
    
    directory = Path(directory)
    
    preprocess(base_directory=directory)
    
    train(
        model_directory=directory / "model",
        train_path=directory / "train", 
        validation_path=directory / "validation",
        epochs=1
    )
    
    # After training a model, we need to prepare a package just like
    # SageMaker would. This package is what the evaluation script is
    # expecting as an input.
    with tarfile.open(directory / "model.tar.gz", "w:gz") as tar:
        tar.add(directory / "model" / "001", arcname="001")
        
    # def evaluate_models(trained_model_path, tuned_model_path, test_path, output_path):
    evaluate_models(
        trained_model_path=directory / "model",  # Path to the trained model
        tuned_model_path=directory / "model",  # Path to the tuned model (simulated as the same for this test)
        test_path=directory / "test",
        output_path=directory / "evaluation",
    )

    yield directory / "evaluation"
    
    shutil.rmtree(directory)


def test_evaluate_generates_evaluation_report(directory):
    output = os.listdir(directory)
    assert "evaluation.json" in output


def test_evaluation_report_contains_accuracy(directory):
    with open(directory / "evaluation.json", 'r') as file:
        report = json.load(file)
        
    assert "best_model" in report
    assert "best_model_accuracy" in report
    assert "best_model_path" in report
    assert "comparison" in report
    assert "trained_model_accuracy" in report["comparison"]
    assert "tuned_model_accuracy" in report["comparison"]
    

8/8 - 0s - loss: 1.3973 - accuracy: 0.2218 - val_loss: 1.3197 - val_accuracy: 0.2549 - 134ms/epoch - 17ms/step
2/2 [==============================] - 0s 874us/step
Validation accuracy: 0.2549019607843137
INFO:tensorflow:Assets written to: /var/folders/5k/bjy1b7pd2wxctw1dtvx0l2fc0000gn/T/tmp0csvtmys/model/001/assets
1/1 [==============================] - 0s 7ms/step
.8/8 - 0s - loss: 1.0948 - accuracy: 0.3138 - val_loss: 1.0675 - val_accuracy: 0.4510 - 137ms/epoch - 17ms/step
2/2 [==============================] - 0s 972us/step
Validation accuracy: 0.45098039215686275
INFO:tensorflow:Assets written to: /var/folders/5k/bjy1b7pd2wxctw1dtvx0l2fc0000gn/T/tmpkvngy4dx/model/001/assets
1/1 [==============================] - 0s 7ms/step
.
2 passed in 1.44s


In [10]:
# | code: true
# | output: false

from sagemaker.workflow.steps import TuningStep
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, ContinuousParameter, CategoricalParameter
from sagemaker.tensorflow import TensorFlowProcessor

evaluation_processor = TensorFlowProcessor(
    base_job_name="evaluation-processor",
    image_uri=config["image"],
    framework_version=config["framework_version"],
    py_version=config["py_version"],
    instance_type=config["instance_type"],
    instance_count=1,
    role=role,
    sagemaker_session=config["session"],
)

# target_metric_name = "val_accuracy"
# objective_type="Maximize",
target_metric_name = "loss"
objective_type="Minimize",

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name=target_metric_name,
    objective_type=objective_type,
    hyperparameter_ranges={
        'learning_rate': ContinuousParameter(0.001, 0.2),
        "epochs": IntegerParameter(10, 50),
    },
    metric_definitions=[{"Name": target_metric_name, "Regex": f"{target_metric_name}: ([0-9\\.]+)"}],
    max_jobs=3,
    max_parallel_jobs=3,
)

tune_model_step = TuningStep(
    name="tune-model",
    step_args=tuner.fit(
        inputs={
            "train": TrainingInput(
                s3_data=preprocessing_step.properties.ProcessingOutputConfig.Outputs[
                    "train"
                ].S3Output.S3Uri,
                content_type="text/csv",
            ),
            "validation": TrainingInput(
                s3_data=preprocessing_step.properties.ProcessingOutputConfig.Outputs[
                    "validation"
                ].S3Output.S3Uri,
                content_type="text/csv",
            ),
        },
    ),
    cache_config=cache_config,
)
model_assets = train_model_step.properties.ModelArtifacts.S3ModelArtifacts

tuned_model_assets = tune_model_step.get_top_model_s3_uri(
    top_k=0, s3_bucket=config["session"].default_bucket()
)

/Users/jerryobex/proj/mlschool/.venv/lib/python3.9/site-packages/sagemaker/workflow/pipeline_context.py:297: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [12]:
# | code: true
# | output: true

from sagemaker.workflow.model_step import ModelStep
from sagemaker.model_metrics import ModelMetrics, MetricsSource
from sagemaker.tensorflow.model import TensorFlowModel
from sagemaker.workflow.functions import Join
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep

from sagemaker.workflow.properties import PropertyFile

evaluation_report = PropertyFile(
    name="evaluation-report", output_name="evaluation", path="evaluation.json"
)

evaluate_model_step = ProcessingStep(
    name="evaluate-model",
    # evaluate_models(
    #     trained_model_path="/opt/ml/processing/trained_model/",
    #     tuned_model_path="/opt/ml/processing/tuned_model/",
    #     test_path="/opt/ml/processing/test/",
    #     output_path="/opt/ml/processing/evaluation/"
    # )
    step_args=evaluation_processor.run(
        inputs=[
            # The first input is the test split that we generated on
            # the first step of the pipeline when we split and
            # transformed the data.
            ProcessingInput(
                source=preprocessing_step.properties.ProcessingOutputConfig.Outputs[
                    "test"
                ].S3Output.S3Uri,
                destination="/opt/ml/processing/test",
            ),
            # The second input is the model that we generated on
            # the Training Step.
            ProcessingInput(
                source=model_assets,
                destination="/opt/ml/processing/model",
            ),
            # The third input is the tuned model that we generated on
            # the Tuning Step.
            ProcessingInput(
                source=tuned_model_assets,
                destination="/opt/ml/processing/model",
            ),
        ],
        outputs=[
            # The output is the evaluation report that we generated
            # in the evaluation script.
            ProcessingOutput(
                output_name="evaluation", source="/opt/ml/processing/evaluation"
            ),
        ],
        code=f"{CODE_FOLDER}/evaluation.py",
    ),
    property_files=[evaluation_report],
    cache_config=cache_config,
)

MODEL_PACKAGE_GROUP = "penguins"

tensorflow_model = TensorFlowModel(
    model_data=model_assets,
    framework_version=config["framework_version"],
    sagemaker_session=config["session"],
    role=role,
)

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=Join(
            on="/",
            values=[
                evaluate_model_step.properties.ProcessingOutputConfig.Outputs["evaluation"].S3Output.S3Uri,
                "evaluation.json",
            ],
        ),
        content_type="application/json",
    )
)

register_model_step = ModelStep(
    name="register-model",
    step_args=tensorflow_model.register(
        model_package_group_name=MODEL_PACKAGE_GROUP,
        approval_status="Approved",
        model_metrics=model_metrics,
        content_types=["text/csv"],
        response_types=["application/json"],
        inference_instances=[config["instance_type"]],
        transform_instances=[config["instance_type"]],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="TENSORFLOW",
        framework_version=config["framework_version"],
    ),
)

# Initialize a boto3 client for Lambda
lambda_client = boto3.client('lambda')

# Define the payload for the Lambda function
payload = {
    "best_model_accuracy_default": 0.5
}

# Invoke the Lambda function
response = lambda_client.invoke(
    FunctionName="arn:aws:lambda:us-east-2:992382774880:function:MlSchoolStack-ModelAccuraccyLambdaFunctionA882847A-axbrhl7xbLiM",
    InvocationType="RequestResponse",
    Payload=json.dumps(payload),
)

# Process the response from Lambda
response_payload = json.loads(response['Payload'].read())
response_payload_dict = json.loads(response_payload['body'])
print(response_payload_dict)

# Use the output from Lambda as the accuracy threshold
accuracy_threshold = response_payload_dict["best_model_accuracy"] if "best_model_accuracy" in response_payload_dict else 0.5

latest_model_accuracy = JsonGet(
    step_name=evaluate_model_step.name,
    property_file=evaluation_report,
    json_path="metrics.accuracy.value",
)

fail_step = FailStep(
    name="fail",
    error_message=Join(
        on=" ",
        values=[
            "Execution failed because the model's accuracy was lower than",
            accuracy_threshold,
        ],
    ),
)

better_model_condition = ConditionGreaterThanOrEqualTo(
    left=latest_model_accuracy, right=accuracy_threshold,
)

register_model_condition_step = ConditionStep(
    name="check-model-accuracy",
    conditions=[better_model_condition],
    if_steps=[register_model_step] if not LOCAL_MODE else [],
    else_steps=[fail_step],
)

publish_best_model_step = ModelStep(
    name="register-model",
    step_args=tensorflow_model.register(
        model_package_group_name=MODEL_PACKAGE_GROUP,
        approval_status="Approved",
        model_metrics=model_metrics,
        content_types=["text/csv"],
        response_types=["application/json"],
        inference_instances=[config["instance_type"]],
        transform_instances=[config["instance_type"]],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="TENSORFLOW",
        framework_version=config["framework_version"],
    ),
)

finish_evaluation_condition_step = ConditionStep(
    name="publish-most-accurate-model",
    conditions=[better_model_condition],
    if_steps=[publish_best_model_step] if not LOCAL_MODE else [],
    else_steps=[fail_step],
)


Failed to retrieve accuracy metric from the latest model package.


TypeError: string indices must be integers

In [ ]:
# | code: true
# | output: false
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig

pipeline_definition_config = PipelineDefinitionConfig(use_custom_job_prefix=True)


session3_pipeline = Pipeline(
    name="session3-pipeline",
    parameters=[dataset_location, epochs, accuracy_threshold],
    steps=[
        preprocessing_step,
        # tune_model_step if USE_TUNING_STEP else train_model_step,
        train_model_step,
        tune_model_step,
        evaluate_model_step,
        register_model_condition_step,
        finish_evaluation_condition_step,
    ],
    pipeline_definition_config=pipeline_definition_config,
    sagemaker_session=config["session"],
)

session3_pipeline.upsert(role_arn=role)

: 

We can now start the pipeline:

#| hide

<div class="alert" style="background-color:#0066cc;"><strong>Note:</strong> 
    The <code>%%script</code> cell magic is a convenient way to prevent the notebook from executing a specific cell. If you want to run the cell, comment out the line containing the <code>%%script</code> cell magic.
</div>


In [ ]:
#%%script false --no-raise-error
#| eval: false
#| code: true
#| output: false

session3_pipeline.start()

: 

### Assignments

-   <span style="padding:4px; line-height:30px; background-color: #f2a68a; color: #000;"><strong>Assignment 3.1</strong></span> The evaluation script computes the accuracy of the model and exports it as part of the evaluation report. Extend the evaluation report by adding the precision and the recall of the model on each one of the classes.

-   <span style="padding:4px; line-height:30px; background-color: #f2a68a; color: #000;"><strong>Assignment 3.2</strong></span> Extend the evaluation script to test the model on each island separately. The evaluation report should contain the accuracy of the model on each island and the overall accuracy.

-   <span style="padding:4px; line-height:30px; background-color: #f2a68a; color: #000;"><strong>Assignment 3.3</strong></span> The Condition Step uses a hard-coded threshold value to determine if the model's accuracy is good enough to proceed. Modify the code so the pipeline uses the accuracy of the latest registered model version as the threshold. We want to register a new model version only if its performance is better than the previous version we registered.

-   <span style="padding:4px; line-height:30px; background-color: #f2a68a; color: #000;"><strong>Assignment 3.4</strong></span> The current pipeline uses either a Training Step or a Tuning Step to build a model. Modify the pipeline to use both steps at the same time. The evaluation script should evaluate the model coming from the Training Step and the best model coming from the Tuning Step and output the accuracy and location in S3 of the best model. You should modify the code to register the model assets specified in the evaluation report.

-   <span style="padding:4px; line-height:30px; background-color: #f2a68a; color: #000;"><strong>Assignment 3.5</strong></span> Pipeline steps can encounter exceptions. In some cases, retrying can resolve these issues. For this assignment, configure the Processing Step so it automatically retries the step a maximum of 5 times if it encounters an `InternalServerError`. Check the [Retry Policy for Pipeline Steps](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-retry-policy.html) documentation for more information.